# Preprocessamento de Dados de Reclamações de Aeroporto

Preparação dos dados para modelagem, incluindo limpeza, transformação, balanceamento e divisão em conjuntos de treino e teste.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
df = pd.read_excel('data/raw/claims-data-2015-as-of-feb-9-2016.xlsx')
df

,Claim Number,Date Received,Incident D,Airport Code,Airport Name,Airline Name,Claim Type,Claim Site,Item Category,Close Amount,Disposition
0,2015060222904,2015-05-28,2015-05-21,ABE,"Lehigh Valley International Airport, Allentown",Allegiant Air,Property Damage,Checkpoint,Personal Electronics,0,Deny
1,2015082725115,2015-08-21,2015-08-01,ABE,"Lehigh Valley International Airport, Allentown",-,Property Damage,Checked Baggage,Sporting Equipment & Supplies,180,Settle
2,2015102326219,2015-10-07,2015-10-02,ABE,"Lehigh Valley International Airport, Allentown",Allegiant Air,Passenger Property Loss,Checked Baggage,Clothing,60,Approve in Full
3,2015122027695,2015-10-27,2015-10-19,ABE,"Lehigh Valley International Airport, Allentown",Allegiant Air,Property Damage,Checkpoint,Personal Accessories,-,-
4,2015012220065,2015-01-22,2014-12-23,ABI,Abilene Regional,American Airlines,Property Damage,Checked Baggage,Home Decor,0,Deny
...,...,...,...,...,...,...,...,...,...,...,...
8662,2015120427297,2015-11-20,2015-10-16,-,-,-,Property Damage,Checked Baggage,"Baggage/Cases/Purses; Books, Magazines & Other...",-,-
8663,2015123027969,2015-12-17,2015-12-02,-,-,-,Property Damage,Checked Baggage,Audio/Video; Home Decor,-,-
8664,2016010428072,2015-12-22,2015-12-20,-,-,-,Passenger Property Loss,Checked Baggage,Clothing,-,-
8665,2016011328300,2015-12-30,2015-12-28,-,-,-,Passenger Property Loss,Checked Baggage,Tools & Home Improvement Supplies,-,-


## Conversão de datas e criação de variáveis

Transformação das colunas de data para datetime e criação da coluna 'days_to_claim', representando o tempo entre incidente e reclamação.

In [3]:
df['Incident D'] = pd.to_datetime(df['Incident D'])
df['Date Received'] = pd.to_datetime(df['Date Received'])

df['days_to_claim'] = (df['Date Received'] - df['Incident D']).dt.days


## Remoção de colunas irrelevantes

Exclusão de colunas que não serão utilizadas na modelagem, mantendo apenas as variáveis informativas.

In [4]:
columns_to_drop = ['Claim Number', 'Airport Code', 'Airport Name', 'Date Received', 'Incident D', 'Close Amount']
df = df.drop(columns=columns_to_drop).copy()

## Agrupamento de categorias menos frequentes

Redução da cardinalidade das colunas categóricas, mantendo apenas os valores mais frequentes e substituindo os demais por 'Other'.

In [5]:
top_items = df['Item Category'].value_counts().head(10).index
df['Item Category'] = df['Item Category'].apply(lambda x: x if x in top_items else 'Other')

top_claim_types = df['Claim Type'].value_counts().head(10).index
df['Claim Type'] = df['Claim Type'].apply(lambda x: x if x in top_claim_types else 'Other')

top_claim_sites = df['Claim Site'].value_counts().head(10).index
df['Claim Site'] = df['Claim Site'].apply(lambda x: x if x in top_claim_sites else 'Other')

## Codificação da variável alvo

Conversão da coluna 'Disposition' em variável binária: 1 para 'Deny' e 0 para outras disposições.

In [6]:
df['Disposition'] = df['Disposition'].map(lambda x: 1 if x == 'Deny' else 0)
df['Disposition'].value_counts()

Disposition
0    5093
1    3574
Name: count, dtype: int64

## Limpeza de valores e filtragem

Remoção de espaços em branco e linhas com valores ausentes ou inválidos; filtragem das principais companhias aéreas.

In [7]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df[~df.isin(['-']).any(axis=1)].copy()

In [8]:
top_airlines = df['Airline Name'].value_counts().head(5).index
df = df[df['Airline Name'].isin(top_airlines)].copy()

In [9]:
df

,Airline Name,Claim Type,Claim Site,Item Category,Disposition,days_to_claim
4,American Airlines,Property Damage,Checked Baggage,Other,1,30
5,American Airlines,Passenger Property Loss,Checked Baggage,Baggage/Cases/Purses,0,22
6,American Airlines,Passenger Property Loss,Checked Baggage,Personal Electronics,1,32
7,Delta Air Lines,Passenger Property Loss,Checked Baggage,Clothing,1,46
9,Southwest Airlines,Passenger Property Loss,Checked Baggage,Other,0,94
...,...,...,...,...,...,...
8568,USAir,Passenger Property Loss,Checked Baggage,Other,0,137
8569,American Airlines,Passenger Property Loss,Checkpoint,Other,1,1
8645,USAir,Passenger Property Loss,Checked Baggage,Jewelry & Watches,0,51
8646,Delta Air Lines,Property Damage,Checked Baggage,Baggage/Cases/Purses,0,40


## Separação em treino e teste

Divisão dos dados em conjuntos de treino e teste, mantendo a proporção das classes na variável alvo.

In [10]:
from sklearn.model_selection import train_test_split

X = df.drop('Disposition', axis=1)
y = df['Disposition']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

## Identificação de colunas numéricas e categóricas

Definição das colunas numéricas e categóricas para aplicação de transformações específicas.

In [11]:
num_cols = X_train.select_dtypes(include='number').columns.tolist()
cat_cols = X_train.select_dtypes(include='object').columns.tolist()

## Construção de pipelines de pré-processamento

Criação de pipelines para imputação, escalonamento e codificação one-hot, aplicados separadamente em colunas numéricas e categóricas.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE


num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

full_preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])


## Aplicação das transformações

Transformação dos conjuntos de treino e teste utilizando os pipelines definidos.

In [13]:
X_train_transformed = full_preprocessor.fit_transform(X_train)
if hasattr(X_train_transformed, "toarray"):  
    X_train_transformed = X_train_transformed.toarray()

X_test_transformed = full_preprocessor.transform(X_test)
if hasattr(X_test_transformed, "toarray"):
    X_test_transformed = X_test_transformed.toarray()

## Balanceamento do conjunto de treino

Aplicação do SMOTE para balancear as classes no conjunto de treino.

In [14]:
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_transformed, y_train)


## Conversão para DataFrame e salvamento

Transformação dos arrays resultantes em DataFrames com nomes de features e salvamento dos conjuntos de treino e teste processados em CSV.

In [15]:
feature_names = (
    num_cols +
    list(full_preprocessor.named_transformers_['cat'].get_feature_names_out(cat_cols))
)

X_train_df = pd.DataFrame(X_train_res, columns=feature_names)
X_test_df = pd.DataFrame(X_test_transformed, columns=feature_names)

X_train_df.head(10)

,days_to_claim,Airline Name_American Airlines,Airline Name_Delta Air Lines,Airline Name_Southwest Airlines,Airline Name_UAL,Airline Name_USAir,Claim Type_Complaint,Claim Type_Passenger Property Loss,Claim Type_Personal Injury,Claim Type_Property Damage,...,Claim Site_Other,Item Category_Baggage/Cases/Purses,Item Category_Clothing,Item Category_Computer & Accessories,Item Category_Cosmetics & Grooming,Item Category_Jewelry & Watches,Item Category_Other,Item Category_Personal Accessories,Item Category_Personal Electronics,Item Category_Travel Accessories
0,-0.471700,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.230930,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.115176,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.366363,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.291123,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,-0.486748,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.185786,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.145272,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,-0.140642,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,-0.471700,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [16]:
X_train_df.to_csv("data/processed/X_train.csv", index=False)
y_train_res.to_csv("data/processed/y_train.csv", index=False)

X_test_df.to_csv("data/processed/X_test.csv", index=False)
y_test.to_csv("data/processed/y_test.csv", index=False)